In [5]:
from bs4 import BeautifulSoup
import requests
import time

#ヤフーニュースの情報を取得
def yahoo_news_scraping(topic):
    url = 'https://news.yahoo.co.jp/'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    list_item = soup.find_all('li')
    #もしかしたらエラーの原因になるかも
    list_item = list_item[12:21]
    
    # <a>要素からトピックを取得
    a_contents = [li.a for li in list_item]
    
    articles_dict = []
    topic = topic 
    #トピックのURL取得
    for i, a_content in enumerate(a_contents):
        if topic in a_content.text:
            categories_num = i
    
    a_content = a_contents[categories_num]
    
    topic_url = a_content['href']
    URL = url + topic_url
    
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, "html.parser")
    #クラス名が変わるので注意！！
    elements = soup.select('.sc-fXazdy.UjHkE')
    
    elements = elements[:5]
    
    news_urls = []
    news_titles = []
    #ニュースのURLとタイトルを取得
    for element in elements:
        url = element.a['href']
        title = element.text[1:]
        news_urls.append(url)
        news_titles.append(title)
            
    news_text = []
    #ニュースの本文を取得
    for news_url in news_urls:
        r = requests.get(news_url)
        soup = BeautifulSoup(r.text, "html.parser")
        #クラス名が変わるので注意！！ 
        elements = soup.select('.sc-ksXiki.dXXEDb.yjSlinkDirectlink.highLightSearchTarget')
    
        text = ""
    
        #ニュース記事の本文が配列で区切られていた場合
        for i in range(0, 8):
            try:
                if (i == 0):
                    text = elements[0].text
                else:
                    text += elements[i].text
            except:
                pass
    
        #ページが1つだけのとき、エラーが出るため
        try:
            #クラス名が変わるので注意！！ （取得例：２ページ)
            page_element = soup.select('.sc-cpUASM.bqNXll')
            page = page_element[0].text         
            page_str = ''.join(filter(str.isdigit, page))
            page_num = int(page_str)
        except:
            page_num = 1
            
        #ページ数が複数のとき、ページ遷移し本文を取得
        if (page_num > 1):
            for i in range(2, page_num + 1):
                news_url_page = news_url + "?page=" + str(page_num)
                r = requests.get(news_url_page)
                soup = BeautifulSoup(r.text, "html.parser")
                #クラス名が変わるので注意！！ 
                elements = soup.select('.sc-iMCRTP.ePfheF.yjSlinkDirectlink.highLightSearchTargett')
                #ニュース記事の本文が配列で区切られていた場合
                for i in range(0, 8):
                    try:
                        text += elements[i].text
                    except:
                        pass
                        
        news_text.append(text)
    
        time.sleep(0.5)

    articles = []
    for url, title, text in zip(news_urls, news_titles, news_text):
        article = {
            "title" : title,
            "URL" : url,
            "text" : text
        }
        articles.append(article)

    return articles

In [6]:
a = yahoo_news_scraping('経済')

In [7]:
a

[{'title': '3人以上の子供がいる家庭の大学無償化へテレ東BIZ12/7(木)10:33',
  'URL': 'https://news.yahoo.co.jp/articles/e68a59a43c8f0142a913ec93a2e9e35ac0292bd3',
  'text': '政府は子育て支援策として、３人以上子供がいる家庭について大学の授業料などを無償にする方向で調整に入りました。所得制限は設けない方針です。今月中に決定する「こども未来戦略」に盛り込むことを目指します。'},
 {'title': 'トヨタ新型「和製スポーツカー」実車公開！ めちゃ「低く」「ワイド」なフォルムに“熱視線”！ 欧州で初展示くるまのニュース12/7(木)12:10',
  'URL': 'https://news.yahoo.co.jp/articles/8146257898e56c3dfa0b70cbbb8311ac8cc31d1a',
  'text': '\u3000トヨタの欧州法人「トヨタモーターヨーロッパ（TME）」は2023年12月4日（現地時間）に最新の電動モデルや技術を発表するイベント「Kenshiki」を開催しました。\n\u3000\n\u3000数多くの発表が行われたなかには、欧州初披露となる新型スポーツカー「FT-Se」の姿もありました。\n\n【画像】めちゃカッコいい！ トヨタ新型「スポーツカー」を画像で見る（30枚以上）\u3000トヨタではグローバルビジョン「Mobility for All」を掲げ、2050年までに世界全体で完全なカーボンニュートラルを実現する取り組みを実施しており、なかでも欧州では世界に先駆け2040年までの達成を目指しています。\n\n\u3000こうした将来の取り組み実現に向けTMEは、この先数年の間に重要な役割を果たすBEV（バッテリーEV：電気自動車）とFCEV（水素燃料電池車）を中心にイベントで披露しました。\n\n\u3000そのうちの1台が新型スポーツカーのFT-Seです。\n\n\u3000FT-Seは、2023年10月25日より東京で開催された「ジャパンモビリティショー2023（JMS2023）」で世界初公開されたコンセプトモデル。\n\n\u3000TOYOTA GAZOO Racingが取り組